In [11]:
from new_model import *
from tqdm import tqdm

In [12]:
seed = 777
random.seed(seed)
np.random.seed(seed)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(seed)
if device == 'cuda':
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## 1. CIFAR 10 데이터셋 다운로드

In [3]:
#Data 다운로드 경로지정
DATA_ROOT = "../data_cifar10"
batch_size = 128


#Data Process 
transform_train = transforms.Compose([transforms.RandomCrop(32, padding=4),transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) 

transform_val = transforms.Compose([transforms.ToTensor(), 
                                    transforms.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010))])

transform_test = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010))])

train_CIFAR10 = torchvision.datasets.CIFAR10(root='../data', train=True, download=True, transform=transform_train)

test_CIFAR10 = torchvision.datasets.CIFAR10(root='../data', train=False, download=True, transform=transform_test)

num_train = int(1.0 * len(train_CIFAR10) * 95 / 100)
num_val = len(train_CIFAR10) - num_train
train_CIFAR10, val_CIFAR10 = torch.utils.data.random_split(train_CIFAR10, [num_train, num_val])

train_loader = torch.utils.data.DataLoader(train_CIFAR10, batch_size=batch_size, shuffle=True, num_workers=2) #num_workers는 데이터 로드시 sub process 몇개 쓸거냐 
val_loader = torch.utils.data.DataLoader(val_CIFAR10, batch_size=batch_size,shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_CIFAR10, batch_size=batch_size, shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


## 2. CIFAR 100 데이터셋 다운로드(참고)

In [13]:
#Data 다운로드 경로지정
DATA_ROOT = "../data_cifar100"
batch_size = 128

#Data Process
transform_train = transforms.Compose([transforms.RandomCrop(32, padding=4),transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) 

transform_val = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) 

transform_test = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) 

train_CIFAR100 = torchvision.datasets.CIFAR100(root='../data', train=True, download=True, transform=transform_train)

test_CIFAR100 = torchvision.datasets.CIFAR100(root='../data', train=False, download=True, transform=transform_test)

num_train = int(1.0 * len(train_CIFAR100) * 95 / 100)
num_val = len(train_CIFAR100) - num_train
train_CIFAR100, val_CIFAR100 = torch.utils.data.random_split(train_CIFAR100, [num_train, num_val])

train_loader = torch.utils.data.DataLoader(train_CIFAR100, batch_size=batch_size, shuffle=True, num_workers=2) #num_workers는 데이터 로드시 sub process 몇개 쓸거냐 
val_loader = torch.utils.data.DataLoader(val_CIFAR100, batch_size=batch_size,shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_CIFAR100, batch_size=batch_size, shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


## 4. train_model과 test_model 저장

In [14]:
def train_model(model, mode):    
    # total number of training epochs
    EPOCHS = 200
    CHECKPOINT_PATH = "./saved_model"
    
    best_val_acc = 0
    current_learning_rate = INITIAL_LR

    print("==> Training starts!")

    start = time.time()
    optimizer = torch.optim.SGD(model.parameters(), lr=INITIAL_LR, momentum=MOMENTUM, weight_decay=REG)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones = [60, 120, 160], gamma = 0.2)
    
    for i in tqdm(range(0, EPOCHS)):
        model.train()
        
        # this help you compute the training accuracy
        total_examples = 0
        correct_examples = 0
        train_loss = 0

        
        '''
        train loop
        '''
        # Train the model for 1 epoch.
        for batch_idx, (inputs, targets) in enumerate(train_loader):
            inputs, targets = inputs.to(device).float(), targets.to(device).long() #inputs과 targets는 gpu로 계산

            # compute the output and loss
            y_preds = model(inputs)        
            loss = criterion(y_preds,targets)
            train_loss += loss.item()

            # zero the gradient
            optimizer.zero_grad()

            # backpropagation
            loss.backward()

            # apply gradient and update the weights
            optimizer.step()
            


            # count the number of correctly predicted samples in the current batch
            y_preds_class = torch.argmax(y_preds, dim=1)
            correct_examples += (targets == y_preds_class).sum().item()
            total_examples += targets.size(0)
        
        # scheduler
        scheduler.step()
        avg_loss_tr = train_loss / len(train_loader) ###
        avg_acc_tr = correct_examples / total_examples ### 

        
                                                      
        '''
        validation loop
        '''
                                                      
        # switch to eval mode
        model.eval()
        total_examples = 0
        correct_examples = 0
        val_loss = 0 # again, track the validation loss if you want
        
        # disable gradient during validation, which can save GPU memory
        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(val_loader):
                inputs, targets = inputs.to(device).float(), targets.to(device).long()

                # compute the output and loss
                y_preds = model(inputs)        
                loss = criterion(y_preds,targets)
                val_loss += loss.item()

                # count the number of correctly predicted samples in the current batch
                y_preds_class = torch.argmax(y_preds, dim=1)
                correct_examples += (targets == y_preds_class).sum().item()
                total_examples += targets.size(0)

        avg_loss_val = val_loss / len(val_loader)
        avg_acc_val = correct_examples / total_examples

                                                      
        # save the model checkpoint
        if avg_acc_val > best_val_acc :
            best_val_acc = avg_acc_val
            if not os.path.exists(CHECKPOINT_PATH):
                os.makedirs(CHECKPOINT_PATH)

            state = {'state_dict': model.state_dict(),
                     'epoch': i,
                     'lr': current_learning_rate}
            
            torch.save(state, os.path.join(CHECKPOINT_PATH, '{}.pth'.format(mode)))
        
        if i % 10 == 9 :  #epoch를 5개씩 출력해서 봐보겠다
            end = time.time()
            diff_time = round(end - start,2)
            print("Epoch %d:" %(i+1), f"progress time is {diff_time} sec")
            print("Training loss: %.4f, Training accuracy: %.4f" %(avg_loss_tr, avg_acc_tr)) 
            print("Validation loss: %.4f, Validation accuracy: %.4f" % (avg_loss_val, avg_acc_val))
        time.sleep(0.1)
    print(f"==> Optimization finished! Best validation accuracy: {best_val_acc:.4f}")                                    

    return model #결국 history 변수를 지정해서 epoch 1개씩 loss와 accuracy값을 저장하려고 return 해야됨

In [15]:
def test_model(model):
    model.to(device)
    model.eval()

    total_examples = 0
    correct_examples = 0
    softmax = torch.nn.Softmax(dim=1)

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            pred = model(inputs)
            total_examples += inputs.shape[0]

            out = softmax(pred)
            out = torch.max(out, 1)

            correct_examples += torch.sum(targets==out[1]).cpu().data.numpy().tolist()

    avg_acc = correct_examples / total_examples
    print("Total examples is {}, correct examples is {}; Test accuracy: {}".format(total_examples, correct_examples, avg_acc))

In [16]:
INITIAL_LR = 0.1

# momentum for optimizer
MOMENTUM = 0.9

# L2 regularization strength
REG = 5e-4

# Define loss
criterion = nn.CrossEntropyLoss()

## 5. Wide_resnet model을 train 시켜보자
- net : 28x1, 28x2, 28x4, 28x10

## CIFAR10

In [7]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model = Wide_ResNet(28, 2, 0.3, 10).to(device)
print("The model is deployed to", device)

| Wide-Resnet 28x2
The model is deployed to cuda:1


In [8]:
train_model(model, "resnet28x2_cifar10")

==> Training starts!


  5%|▌         | 10/200 [03:44<1:11:39, 22.63s/it]

Epoch 10: progress time is 224.32 sec
Training loss: 0.5250, Training accuracy: 0.8198
Validation loss: 0.9001, Validation accuracy: 0.7096


 10%|█         | 20/200 [07:33<1:08:27, 22.82s/it]

Epoch 20: progress time is 453.14 sec
Training loss: 0.4191, Training accuracy: 0.8569
Validation loss: 1.0043, Validation accuracy: 0.7032


...

 95%|█████████▌| 190/200 [1:12:20<03:47, 22.72s/it]

Epoch 190: progress time is 4340.18 sec
Training loss: 0.0040, Training accuracy: 0.9999
Validation loss: 0.1786, Validation accuracy: 0.9508


100%|██████████| 200/200 [1:16:09<00:00, 22.85s/it]

Epoch 200: progress time is 4569.54 sec
Training loss: 0.0035, Training accuracy: 0.9998
Validation loss: 0.1725, Validation accuracy: 0.9484
==> Optimization finished! Best validation accuracy: 0.9552


Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(32, 32, kernel_size=(3, 3)

## CIFAR100

In [20]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = Wide_ResNet(28, 2, 0.3, 100).to(device)
print("The model is deployed to", device)

| Wide-Resnet 28x2
The model is deployed to cuda:0


In [21]:
train_model(model, "resnet28x2_cifar100")

==> Training starts!


  5%|▌         | 10/200 [03:58<1:17:13, 24.39s/it]

Epoch 10: progress time is 238.76 sec
Training loss: 1.6447, Training accuracy: 0.5396
Validation loss: 2.0676, Validation accuracy: 0.4512


 10%|█         | 20/200 [08:07<1:14:53, 24.96s/it]

Epoch 20: progress time is 487.41 sec
Training loss: 1.3349, Training accuracy: 0.6189
Validation loss: 1.8437, Validation accuracy: 0.4924


...

 95%|█████████▌| 190/200 [1:17:17<04:02, 24.29s/it]

Epoch 190: progress time is 4637.55 sec
Training loss: 0.0261, Training accuracy: 0.9990
Validation loss: 1.0813, Validation accuracy: 0.7284


100%|██████████| 200/200 [1:21:22<00:00, 24.41s/it]

Epoch 200: progress time is 4882.08 sec
Training loss: 0.0260, Training accuracy: 0.9990
Validation loss: 1.0434, Validation accuracy: 0.7440
==> Optimization finished! Best validation accuracy: 0.7552


Wide_ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
  (M_relu): M_relu()
  (layer1): Sequential(
    (0): M_BasicBlock(
      (conv1): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu1): M_relu()
      (dropout): Dropout(p=0.3, inplace=False)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (M_relu2): M_relu()
      (shortcut): Sequential(
        (0): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): M_BasicBlock(
      (conv1): Conv2d(32, 32, kernel_size=(3, 3)

In [22]:
send_email('resnet28x2_cifar100 finished')

## test_model을 통해 결과확인

In [10]:
#wide_resnet 28x1의 결과값
checkpoint = torch.load("./saved_model/resnet28x2_cifar10.pth")
model.load_state_dict(checkpoint['state_dict'])
test_model(model)

Total examples is 10000, correct examples is 9491; Test accuracy: 0.9491


In [23]:
#wide_resnet 28x2의 결과값
checkpoint = torch.load("./saved_model/resnet28x2_cifar100.pth")
model.load_state_dict(checkpoint['state_dict'])
test_model(model)

Total examples is 10000, correct examples is 7371; Test accuracy: 0.7371
